# 3-5. Action Recognition

## DataSet

### 1. [Kinetics400/700 Dataset](https://deepmind.com/research/open-source/kinetics)
- 인간-물체 상호작용
- 인간-인간 상호작용
- 650,000 개의 비디오 클립을 제공

<img src='https://miro.medium.com/max/2400/1*k3kk3deV6tFY4lopJ7alJA.png' >


### 2. [UCF101 Dataset](https://www.crcv.ucf.edu/data/UCF101.php)
- YouTube에서 수집한 실제 액션 동영상
  - 인간-물체 상호작용
  - 신체-동작
  - 인간-인간 상호작용
  - 악기 연주
  - 스포츠
- 101개의 카테고리, 13,320 개의 비디오

<img src='https://www.researchgate.net/profile/Khurram_Soomro/publication/233815759/figure/fig1/AS:669565927297037@1536648365593/Sample-frames-for-6-action-classes-of-UCF101.png' width=100%>



### 3. [HMDB51 Dataset](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/)
- 주로 영화에서 수집
- HMDB, Prelinger 아카이브, Youtube, Google 비디오 등 공개 데이터베이스 수집
  - 사람 얼굴 행동
  - 물체를 통한 얼굴 행동
  - 신체움직임
  - 물체 - 사람 상호작용
  - 사람 - 사람 상호작용

<img src='https://www.researchgate.net/publication/290181771/figure/fig4/AS:318089251049478@1452849795740/Examples-from-HMDB51-Kuehne-et-al-2011-dataset-for-a-few-of-51-classes.png' width=100%>

### 4. [The 20BN-something-something-V2 Dataset](https://20bn.com/datasets/something-something)
- 인간이 일상적인 사물로 미리 정의된 기본 동작을 수행하는 행위 정의
  - 220,847개의 영상
  - 무언가를 무언가에 넣기 와 같은 물체-물체 주석 포함
  - 영상의 행위를 설명하는 용도로 사용가능

<img src='https://miro.medium.com/max/600/0*udOMDyKgeiWfN8Aa.' width=100%>

## 라이브러리

In [ ]:
!pip3 install --upgrade mxnet-cu101 > /dev/null
!pip3 install --upgrade gluoncv > /dev/null

tcmalloc: large alloc 1147494400 bytes == 0x396a4000 @  0x7f2786c3f615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
ERROR: distributed 2021.1.1 has requirement dask>=2020.12.0, but you'll have dask 2.12.0 which is incompatible.
ERROR: botocore 1.19.62 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
!pip3 install --upgrade youtube-dl > /dev/null

In [ ]:
def get_youtube_video(youtube_url):
  import youtube_dl

  ydl_opts = {  
    # 'format': 'mp4' 
    'format': 'best[height<=480][ext=mp4]' 
    } 
  with youtube_dl.YoutubeDL(ydl_opts) as ydl: 
      info_dict = ydl.extract_info(youtube_url, download=True)    
      filename = ydl.prepare_filename(info_dict)
  return filename

## i3d action recognition example

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mxnet as mx
from mxnet import gluon, nd, image
from mxnet.gluon.data.vision import transforms
from gluoncv.data.transforms import video
from gluoncv import utils
from gluoncv.model_zoo import get_model


In [ ]:
from gluoncv.utils.filesystem import try_import_decord
decord = try_import_decord()

In [ ]:
url = 'https://github.com/bryanyzhu/tiny-ucf101/raw/master/abseiling_k400.mp4'
video_fname = utils.download(url)

100%|██████████| 782/782 [00:00<00:00, 17280.43KB/s]


In [ ]:
vr = decord.VideoReader(video_fname)

In [ ]:
frame_id_list = range(0, 64, 2)
video_data = vr.get_batch(frame_id_list).asnumpy()
clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]

In [ ]:
list(frame_id_list)

[0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48,
 50,
 52,
 54,
 56,
 58,
 60,
 62]

In [ ]:
len(frame_id_list)

32

###  이미지 보정

In [ ]:
transform_fn = video.VideoGroupValTransform(size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
clip_input = transform_fn(clip_input)
clip_input = np.stack(clip_input, axis=0)
clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))
print('Video data is downloaded and preprocessed.')

Video data is downloaded and preprocessed.


In [ ]:
model_name = 'i3d_nl10_resnet101_v1_kinetics400'
net = get_model(model_name, nclass=400, pretrained=True)

# model_i3d_ucf101 = 'i3d_resnet50_v1_ucf101'
# net_i3d_ucf101 = get_model(model_i3d_ucf101, nclass=101, pretrained=True)

# model_i3d_hmdb51 = 'i3d_resnet50_v1_hmdb51'
# net_i3d_hmdb51 = get_model(model_i3d_hmdb51, nclass=51, pretrained=True)

# model_i3d_sthsthv2 = 'i3d_resnet50_v1_sthsthv2'
# net_i3d_sthsthv2 = get_model(model_i3d_sthsthv2, nclass=174, pretrained=True)
print('%s model is successfully loaded.' % model_name)

538298KB [00:10, 53808.03KB/s]                            


i3d_nl10_resnet101_v1_kinetics400 model is successfully loaded.


In [ ]:
pred = net(nd.array(clip_input))

classes = net.classes
topK = 5
ind = nd.topk(pred, k=topK)[0].astype('int')
print('The input video clip is classified to be')
for i in range(topK):
    print('\t[%s], with probability %.3f.'%
          (classes[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))

The input video clip is classified to be
	[abseiling], with probability 0.911.
	[rock_climbing], with probability 0.068.
	[paragliding], with probability 0.009.
	[playing_bagpipes], with probability 0.001.
	[riding_mule], with probability 0.001.


### 영상 이미지로 저장

In [ ]:
def read_video(filename, output_path, start_ms = 0, end_ms = None, step=1):
  from tqdm.notebook import tqdm
  import os 
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()
  # video_frames = [] 

  if not os.path.exists(output_path):
        os.makedirs(output_path)

  cap = cv2.VideoCapture(filename)

  cap.set(cv2.CAP_PROP_POS_MSEC, start_ms);

  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = cap.get(cv2.CAP_PROP_FPS)
  total_ms = frame_count * fps * 1000
  print('Frame width:', width)
  print('Frame height:', height)
  print('Frame count:', frame_count)
  print('FPS:', fps)
  print('total sec:', int(total_ms/1000))

  capture_count = 0
  
  if end_ms != None:
    time_gap_sec = (end_ms - start_ms)/1000
    pbar_count = round(time_gap_sec*fps)
  else:
    pbar_count = round(frame_count)

  with tqdm(total=pbar_count) as pbar:
    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            cap.release()
            break 
        frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        frame_msec = int(cap.get(cv2.CAP_PROP_POS_MSEC))

        # print('{}. msec:{}, endmsec:{}'.format(frame_no, frame_msec, end_ms)) 
        if end_ms != None and frame_msec >= end_ms:
            break

        # print('{}. msec:{}'.format(frame_no, frame_msec)) 

        # video_frames.append(img)
        cv2.imwrite(os.path.join(output_path, '%05d.jpg'%(frame_no)), img)
        capture_count += 1
        pbar.update(step)
        if step > 1:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no + (step-1));
  
  print('capture_count:', capture_count)

  return width, height, fps, frame_count

## 학습데이터 분석

In [ ]:
from gluoncv import utils
video_path = 'http://crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_GolfSwing_g01_c01.avi'
train_data_swing = utils.download(video_path)

100%|██████████| 378/378 [00:00<00:00, 1310.77KB/s]


In [ ]:
width, height, fps, frame_count = read_video(train_data_swing, './train_data_swing')

Frame width: 320
Frame height: 240
Frame count: 172
FPS: 29.97002997002997
total sec: 5154



capture_count: 172


## 영상 준비

In [ ]:
kpga = get_youtube_video('https://www.youtube.com/watch?v=f3KhsQq7VDw')
print(kpga)

[youtube] f3KhsQq7VDw: Downloading webpage
[download] Destination: [2020 KPGA] 3R 하이라이트 _ 제 63회 KPGA 선수권대회 with A-ONE CC-f3KhsQq7VDw.mp4
[download] 100% of 37.41MiB in 00:01
[2020 KPGA] 3R 하이라이트 _ 제 63회 KPGA 선수권대회 with A-ONE CC-f3KhsQq7VDw.mp4


In [ ]:
width, height, fps, frame_count = read_video(kpga, './kpga')

Frame width: 640
Frame height: 360
Frame count: 18574
FPS: 30.0
total sec: 557220



capture_count: 18574


### 영상 분석 코드


In [ ]:
# video reader 객체 생성
vr = decord.VideoReader(kpga)

In [ ]:
# video reader의 length
video_length = len(vr)
print('frame length:{}'.format(video_length))

frame length:18574


In [ ]:
# 64개의 영상 단위로 분석
window_size = 64
image_size = 64

In [ ]:
from gluoncv.data.transforms import video
transform_fn = video.VideoGroupValTransform(size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [ ]:
print('total frame count : {}'.format(len(vr)))
print('total recognition step : {}'.format(round(len(vr)/window_size)))

total frame count : 18574
total recognition step : 290


### action detect

In [ ]:
def action_recognition_and_print(start_index, clip_input, topK=5):
  pred = net(nd.array(clip_input))

  classes = net.classes 
  ind = nd.topk(pred, k=topK)[0].astype('int')
  print('{} clip is classified to be'.format(start_index))
  for i in range(topK):
      print('\t[%s], with probability %.3f.'%
            (classes[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))



### video reader memory leak으로 인해 아래 코드 사용하지 않음

In [ ]:
# for start_index in range(0, video_length, window_size): 
#   # image_size 64개 확보가 되지않는경우(영상 종료시점 체크)
#   if video_length - window_size < image_size:
#     break
  
#   # 영상분석 이미지 추출
#   frame_id_list = range(0+start_index, 64+start_index, 2)
#   video_data = vr.get_batch(frame_id_list).asnumpy()
#   clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]
  
#   # 이미지 보정
#   clip_input = transform_fn(clip_input)
#   clip_input = np.stack(clip_input, axis=0)
#   clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
#   clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))

#   # 영상 분석
#   action_recognition_and_print(start_index, clip_input)

### 사람이 있는 영상만 분석
- 64의 이미지 묶음 중 처음과 중간 이미지를 분석하여
  - clip_input[0], clip_input[image_size/2]
- 사람이 detect되는 이미지 묶음만 처리하자

### person detect function

In [ ]:
# pose estimation 에서 사용했던 코드 재사용
detector = get_model('yolo3_mobilenet1.0_coco', pretrained=True) 
detector.reset_class(["person"], reuse_weights=['person'])

88993KB [00:01, 72411.66KB/s]                           


In [ ]:
def person_detect(im_fname):   
  from gluoncv import model_zoo, data, utils
  
  x, _ = data.transforms.presets.yolo.load_test(im_fname)

  class_IDs, scores, bounding_boxs = detector(x)

  L = class_IDs.shape[1] 
  for i in range(L):
    if class_IDs[0][i].asscalar() == 0:
      return True
  
  return False
  

In [ ]:
import os

In [ ]:
image_path = './kpga'

In [ ]:
file_list = os.listdir(image_path)
file_list.sort()

In [ ]:
print(video_length)

18574


## video reader 구현.

In [ ]:
# file 목록을 받아, image array list를 리턴한다.
def read_image_batch(frame_id_list):
  from gluoncv import data

  video_data = []
  for index in frame_id_list:
    im_fname = os.path.join(image_path, '%05d.jpg'%(index+1))
    _, img = data.transforms.presets.yolo.load_test(im_fname) 

    video_data.append(img)

  return video_data

### action recognition 함수

In [ ]:
def action_recognition(clip_input, topK=5):
  pred = net(nd.array(clip_input))

  classes = net.classes 
  ind = nd.topk(pred, k=topK)[0].astype('int') 

  result = []
  for i in range(topK):
      # print('[%s], with probability %.3f.'%(classes[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))
      result.append('%s, %.3f.'%(classes[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))

  return result

### 영상분석 dry run...
- 그런데 모든 영상을 분석해야하는건가...

In [ ]:
for start_index in range(0, len(file_list), window_size): 
  # image_size 64개 확보가 되지않는경우(영상 종료시점 체크)
  if video_length - window_size < image_size:
    break 
  
  try:
    # 영상분석 이미지 추출
    frame_id_list = range(0+start_index, 64+start_index, 2) 
    org_image_list = read_image_batch(frame_id_list)
    video_data = np.array(org_image_list)
    clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]
    
    # person detect
    filename1 = os.path.join(image_path, '%05d.jpg'%(start_index+1))
    filename2 = os.path.join(image_path, '%05d.jpg'%(start_index + 32 +1))
    result1 = person_detect(filename1)
    result2 = person_detect(filename2)
    # print('detect result:{}/{}'.format(result1, result2))
    
    if result1 == False and result2 == False:
      continue

    # 이미지 보정
    clip_input = transform_fn(clip_input)
    clip_input = np.stack(clip_input, axis=0)
    clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
    clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))

    # 영상 분석
    action_result = action_recognition(clip_input)
    print(start_index)
    for result_ in action_result:
      print( result_)

  except Exception as ex:
    print(ex)
    continue

320
javelin_throw, 0.525.
hammer_throw, 0.097.
throwing_discus, 0.067.
playing_cricket, 0.057.
shot_put, 0.053.
448
javelin_throw, 0.626.
playing_cricket, 0.069.
throwing_discus, 0.068.
hammer_throw, 0.062.
catching_or_throwing_baseball, 0.033.
576
ski_jumping, 0.107.
bobsledding, 0.090.
skiing_-not_slalom_or_crosscountry-, 0.070.
swimming_backstroke, 0.042.
shredding_paper, 0.029.
704
presenting_weather_forecast, 0.181.
news_anchoring, 0.126.
texting, 0.073.
pumping_gas, 0.052.
answering_questions, 0.032.
768
news_anchoring, 0.261.
texting, 0.130.
presenting_weather_forecast, 0.087.
driving_car, 0.068.
cleaning_windows, 0.056.
832
playing_cricket, 0.457.
golf_driving, 0.138.
golf_putting, 0.117.
catching_or_throwing_baseball, 0.084.
golf_chipping, 0.043.
896
catching_or_throwing_baseball, 0.260.
playing_cricket, 0.147.
triple_jump, 0.079.
golf_putting, 0.075.
long_jump, 0.066.
960
triple_jump, 0.270.
long_jump, 0.189.
playing_cricket, 0.053.
faceplanting, 0.048.
crawling_baby, 0.041.


### 이미지에 text write

In [ ]:
# frame : 이미지
# str : 문자열
# (x, y) : 문자열 표시 좌표
# cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
# 1 : 문자 크기(scale)
# (0, 255, 0) : 색상 (b,g,r)

# cv2.putText(img, '{:s} {:s}'.format(class_name, score),
#                         (xmin, y), cv2.FONT_HERSHEY_SIMPLEX, min(scale/2, 2),
#                         bcolor)

In [ ]:
from gluoncv.utils.filesystem import try_import_cv2
cv2 = try_import_cv2()

In [ ]:
# !rm -rf kpga_detect*

In [ ]:
output_path = './kpga_detect'
if not os.path.exists(output_path):
      os.makedirs(output_path)

### 대표이미지(첫번째 이미지)에 분석내용을 기록하고 저장한다

In [ ]:
for start_index in range(0, len(file_list), window_size): 
  # image_size 64개 확보가 되지않는경우(영상 종료시점 체크)
  if video_length - window_size < image_size:
    break 
  
  try:
    # 영상분석 이미지 추출
    frame_id_list = range(0+start_index, 64+start_index, 2) 
    org_image_list = read_image_batch(frame_id_list)
    video_data = np.array(org_image_list)
    clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]
    
    # person detect
    filename1 = os.path.join(image_path, '%05d.jpg'%(start_index+1))
    filename2 = os.path.join(image_path, '%05d.jpg'%(start_index + 32 +1))
    result1 = person_detect(filename1)
    result2 = person_detect(filename2)
    # print('detect result:{}/{}'.format(result1, result2))
    
    if result1 == False and result2 == False:
      continue

    # 이미지 보정
    clip_input = transform_fn(clip_input)
    clip_input = np.stack(clip_input, axis=0)
    clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
    clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))

    # 영상 분석
    action_result = action_recognition(clip_input)

    
    output_img = org_image_list[0] 
    output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)

    print(start_index)
    cv2.putText(output_img, '%05d'%(start_index),
                    (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                    (255,0,255) , 1, cv2.LINE_AA)
    
    for result_idx, result_ in enumerate( action_result):
      print( result_)
      cv2.putText(output_img, result_,
                        (50, 80 + result_idx*25), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                        (255,0,0) , 1, cv2.LINE_AA)


    

    cv2.imwrite(os.path.join(output_path, '%05d.jpg'%(start_index)), output_img)

  except Exception as ex:
    print(ex)
    continue

320
javelin_throw, 0.525.
hammer_throw, 0.097.
throwing_discus, 0.067.
playing_cricket, 0.057.
shot_put, 0.053.
448
javelin_throw, 0.626.
playing_cricket, 0.069.
throwing_discus, 0.068.
hammer_throw, 0.062.
catching_or_throwing_baseball, 0.033.
576
ski_jumping, 0.107.
bobsledding, 0.090.
skiing_-not_slalom_or_crosscountry-, 0.070.
swimming_backstroke, 0.042.
shredding_paper, 0.029.
704
presenting_weather_forecast, 0.181.
news_anchoring, 0.126.
texting, 0.073.
pumping_gas, 0.052.
answering_questions, 0.032.
768
news_anchoring, 0.261.
texting, 0.130.
presenting_weather_forecast, 0.087.
driving_car, 0.068.
cleaning_windows, 0.056.
832
playing_cricket, 0.457.
golf_driving, 0.138.
golf_putting, 0.117.
catching_or_throwing_baseball, 0.084.
golf_chipping, 0.043.
896
catching_or_throwing_baseball, 0.260.
playing_cricket, 0.147.
triple_jump, 0.079.
golf_putting, 0.075.
long_jump, 0.066.
960
triple_jump, 0.270.
long_jump, 0.189.
playing_cricket, 0.053.
faceplanting, 0.048.
crawling_baby, 0.041.


In [ ]:
def compress_folder(input_path):
  from zipfile import ZipFile
  import os
  from os.path import basename
  from tqdm.notebook import tqdm

  # create a ZipFile object
  with ZipFile('{}.zip'.format(input_path), 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(input_path):
        for filename in tqdm(filenames):
            #create complete filepath of file in directory
            filePath = os.path.join(folderName, filename)
            # Add file to zip
            zipObj.write(filePath, basename(filePath))

In [ ]:
compress_folder('kpga_detect')

# 실습

## 1. 모든 이미지를 저장한다.
- 같은 묶음의 이미지에는 detect 결과를 같은 내용을 출력한다

In [ ]:
output_path = './kpga_detect_all'

In [ ]:
input_path = './kpga'

In [ ]:
import os
file_list = os.listdir(image_path)
file_list.sort()

In [ ]:
def write_recognition_result_image(output_path, start_index, org_image_list, action_result):
  
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()
 
  if not os.path.exists(output_path):
        os.makedirs(output_path)

  for index, img in enumerate(org_image_list): 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # print(start_index)
    cv2.putText(img, '%05d'%(start_index+index),
    (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
    (255,0,255) , 1, cv2.LINE_AA)

    if action_result != None:
      for result_idx, result_ in enumerate( action_result): 
        cv2.putText(img, result_,
        (50, 80 + result_idx*25), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
        (255,0,0) , 1, cv2.LINE_AA)
 
    cv2.imwrite(os.path.join(output_path, '%05d.jpg'%(start_index+index)), img)


In [ ]:
def get_action_recognition_image(image_path, file_list, window_size, image_size):
  from tqdm.notebook import tqdm
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()

  
  with tqdm(total=len(file_list)) as pbar:
    for start_index in range(0, len(file_list), window_size): 
      # image_size 64개 확보가 되지않는경우(영상 종료시점 체크)
      if start_index + window_size > len(file_list) :
        break 
      
      try:
        # 영상분석 이미지 추출
        frame_id_list = range(0+start_index, 64+start_index, 2) 
        org_image_list = read_image_batch(frame_id_list)
        video_data = np.array(org_image_list)
        clip_input = [video_data[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]
        
        # progress bar update
        pbar.update(window_size)

        # person detect
        filename1 = os.path.join(image_path, '%05d.jpg'%(start_index+1))
        filename2 = os.path.join(image_path, '%05d.jpg'%(start_index + 32 +1))
        result1 = person_detect(filename1)
        result2 = person_detect(filename2)
        # print('detect result:{}/{}'.format(result1, result2))
        
        if result1 == False and result2 == False:
          # detect 결과 없이 이미지 번호만 기록된 이미지 저장
          write_recognition_result_image(output_path, start_index, org_image_list, None)
          continue

        # 이미지 보정
        clip_input = transform_fn(clip_input)
        clip_input = np.stack(clip_input, axis=0)
        clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
        clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))

        # 영상 분석
        action_result = action_recognition(clip_input)

        # 영상 분석내용을 이미지 번호와 함께 기록
        write_recognition_result_image(output_path, start_index, org_image_list, action_result)

        # 이미지 기록 -----------------
        # output_img = org_image_list[0] 
        # output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB)

        # print(start_index)
        # cv2.putText(output_img, '%05d'%(start_index),
        #                 (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
        #                 (255,0,255) , 1, cv2.LINE_AA)
        
        # for result_idx, result_ in enumerate( action_result):
        #   print( result_)
        #   cv2.putText(output_img, result_,
        #                     (50, 80 + result_idx*25), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
        #                     (255,0,0) , 1, cv2.LINE_AA)


        

        # cv2.imwrite(os.path.join(output_path, '%05d.jpg'%(start_index)), output_img) 
        # 이미지 기록 -----------------

      except Exception as ex:
        print(ex)
        continue

In [ ]:
get_action_recognition_image(input_path, file_list, 64, 64)

## 2. 저장된 이미지를 이용하여 동영상을 생성한다

In [ ]:
def make_video_file(input_path, output_path, output_filename, width, height, fps, step=1): 
  import os   
  from gluoncv.utils.filesystem import try_import_cv2
  from tqdm.notebook import tqdm 
  cv2 = try_import_cv2()

  #  output path check
  if not os.path.exists(output_path):
          os.makedirs(output_path) 

  # image list
  file_list = os.listdir(input_path)
  file_list.sort()

  # output video codec
  fourcc = cv2.VideoWriter_fourcc(*'FMP4')

  out = cv2.VideoWriter('{}/{}'.format(output_path, output_filename), fourcc, fps/step, (width, height))
  for filename in tqdm(file_list):
  # for filename in tqdm(file_list, desc='{}'.format(output_filename)):
    im_file = '{}/{}'.format(input_path,filename)
    img = cv2.imread(im_file)
    img = cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_AREA)
    out.write(img) 
  out.release()

In [ ]:
input_path = './kpga_detect_all'
output_path = './kpga_detect_video'
output_filename = './kpga_detect.mp4'
width=640
height=360
fps=30.0
step=1

In [ ]:
make_video_file(input_path, output_path, output_filename, width, height, fps, step=step)